In [1]:
import math
import jieba
import psycopg2
import pandas as pd

In [22]:
def gbk(query, conn, doc_len={}, doc_name = {}, top = 10, k=1.2, b=0.75):
    words = jieba.lcut(shorten(query))
    query = []
    for word in words:
        if len(word) == 1:
            continue
        else:
            query.append(word)
    scores = dict()
    idf = dict()
    N = len(doc_len)
    avg_doclen = sum(doc_len.values())/N
    for doc in doc_len:
        scores[doc] = 0
    for key in query:
        dic = get_dictionary(conn, key)
        idf[key] = 1.0 + math.log(N / len(dic))
        for item in dic:
            split = item.split('-')
            juji = int(split[0])
            cnt = int(split[1])
            scores[juji] += idf[key] * (k + 1) * cnt / (k * (1 - b + b*(doc_len[juji] / avg_doclen))+ cnt)
    scores_list = []
    for key in scores:
        scores_list.append((doc_name[key],scores[key]))
    scores_list.sort(key = lambda x: x[1], reverse = True)
    return scores_list[:top]

In [30]:
def get_connection():
    return psycopg2.connect(user = "checker",
                            password = "wangxiaohan",
                            host = "localhost",
                            port = "5432",
                            database = "postgres")

In [4]:
def get_doc_len(con):
    doc_len = dict()
    doc_name = dict()
    cursor = con.cursor()
    doc_len_query = """select * from postgres.ir.text_len"""
    cursor.execute(doc_len_query)
    rows = cursor.fetchall()
    for row in rows:
        doc_len[row[0]] = row[1]
        doc_name[row[0]] = row[2]
    return doc_len, doc_name

In [5]:
def get_dictionary(con, key):
    try:
        cursor = con.cursor()
        reversed_index_query = "select * from postgres.ir.reversed_index where term = '{}'".format(key)
        cursor.execute(reversed_index_query)
        rows = cursor.fetchall()
        return rows[0][2]
    except Exception as e:
        print("执行sql时出错：%s" % (e))
        con.rollback()

In [6]:
def jiebaUtil(conn):
    cursor = conn.cursor()
    query = """select * from postgres.ir.raw_words"""
    cursor.execute(query)
    new_words = cursor.fetchall()
    for key_word in new_words:
        if key_word[1] > 2:
            jieba.add_word(key_word[0], freq = key_word[1]*10, tag = None)

In [7]:
def shorten(s):
    superfluous = ' 啊吧哇呀嘛吗耶？！?!.。诶了阿哈呗啥'
    for i in superfluous:
        s = s.rstrip(i).lower()
    ss= ''.join(reversed(s))
    k = -1
    j = 0
    array = [-1]
    plen = len(ss)
    zeros = 0
    nonze = 0
    while j < plen:
        if k == -1 or ss[j] == ss[k]:
            k += 1
            j += 1
            if k == 0:
                if j != zeros + 1:
                    loop = (int)(nonze/zeros) -1
                    return cut_string(s, plen) if loop<1 else cut_string(s, plen - zeros*loop)
                zeros += 1
            else:
                nonze += 1
            array.append(k)
        else:
            k = array[k]
    return cut_string(s, plen) if zeros==plen else (cut_string(s, zeros) if zeros>1 else cut_string(s, zeros+1)) 
def cut_string(s, lens):
    return s[0: lens]

In [31]:
conn = get_connection()

In [24]:
jiebaUtil()

In [19]:
def jiebaUtil():
    new_words = []
    f = open("new_words_all.txt")
    lines = f.readlines()
    for line in lines:
        split = line.strip().split(',')
        new_words.append((split[0],int(split[1])))
    f.close()
    for key_word in new_words:
        if key_word[1] > 2:
            jieba.add_word(key_word[0], freq = key_word[1]*10, tag = None)

In [26]:
doc_len, doc_name = get_doc_len(conn)

OperationalError: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.


In [25]:
gbk('神抽', conn, doc_len, doc_name)

ZeroDivisionError: division by zero

In [11]:
def get_fanju(con):
    fanju_name = dict()
    name_fanju = dict()
    cursor = con.cursor()
    doc_len_query = """select * from postgres.ir.fanju_id"""
    cursor.execute(doc_len_query)
    rows = cursor.fetchall()
    for row in rows:
        fanju_name[row[0]] = row[1]
        name_fanju[row[1]] = row[0]
    return fanju_name,name_fanju

In [12]:
def distance_ave(conn, clust, target):
    dis = 0
    for i in clust:
        cursor = conn.cursor()
        query = "select avg(distance) from postgres.ir.distance3 where id1 = {} and id2 = {} or id1 = {} and id2 = {}".format(i, target, target, i)
        cursor.execute(query)
        rows = cursor.fetchall()
        dis += rows[0][0]
    return dis / len(clust)

In [13]:
def distance_ave(conn, clust, target):
    dis = 0
    for i in clust:
        cursor = conn.cursor()
        query = "select avg(distance) from postgres.ir.distance3 where id1 = {} and id2 = {} or id1 = {} and id2 = {}".format(i, target, target, i)
        cursor.execute(query)
        rows = cursor.fetchall()
        dis += rows[0][0]
    return dis / len(clust)
def distance_complete(conn, clust, target):
    dis = 0
    for i in clust:
        cursor = conn.cursor()
        query = "select avg(distance) from postgres.ir.distance3 where id1 = {} and id2 = {} or id1 = {} and id2 = {}".format(i, target, target, i)
        cursor.execute(query)
        rows = cursor.fetchall()
        dis = max(rows[0][0],dis)
    return dis
def distance_single(conn, clust, target):
    dis = 1
    for i in clust:
        cursor = conn.cursor()
        query = "select avg(distance) from postgres.ir.distance3 where id1 = {} and id2 = {} or id1 = {} and id2 = {}".format(i, target, target, i)
        cursor.execute(query)
        rows = cursor.fetchall()
        dis = min(rows[0][0],dis)
    return dis

In [14]:
def distance_single(conn, clust, target):
    dis = 1
    for i in clust:
        cursor = conn.cursor()
        query = "select avg(distance) from postgres.ir.distance3 where id1 = {} and id2 = {} or id1 = {} and id2 = {}".format(i, target, target, i)
        cursor.execute(query)
        rows = cursor.fetchall()
        dis = min(rows[0][0],dis)
    return dis

In [15]:
def suggest(conn, clust, fanju_name):
    distance = [1,1,1,1,1,1,1]
    indice = [-1,-1,-1,-1,-1,-1,-1] 
    for i in fanju_name:
        if i in clust:
            continue
        else:
            dis = distance_single(conn,clust,i)
            if dis < distance[0]:
                distance[0] = dis
                indice[0] = i
            dis = distance_complete(conn,clust,i)
            if dis < distance[1]:
                distance[1] = dis
                indice[1] = i
            dis = distance_ave(conn,clust,i)
            maximumd = 0
            maxindex = 0
            for j in range(5):
                if distance[j+2] > maximumd:
                    maximumd = distance[j+2]
                    maxindex = j+2
            if dis < maximumd:
                distance[maxindex] = dis
                indice[maxindex] = i
    res = []
    for i in range(7):
        print(indice[i])
        if indice[i] == -1:
            continue
        if fanju_name[indice[i]] not in res:
            res.append(fanju_name[indice[i]])
    print(res)

In [16]:
fanju_name, name_fanju = get_fanju(conn)

In [17]:
clust = [1026,31,449,604,387]
suggest(conn, clust, fanju_name)

KeyboardInterrupt: 

In [111]:
suggest(conn, clust, fanju_name)

390
1005
390
264
709
1005
1053
['家庭教师HITMANREBORN', '鬼灭之刃', '博人传火影忍者新时代', '犬夜叉', '黑色四叶草']


In [112]:
clust = [59,585,78]
suggest(conn, clust, fanju_name)

76
390
122
390
193
629
817
['LoveLiveSchoolIdolProject', '家庭教师HITMANREBORN', 'TSUKIPRO', '偶像梦幻祭', '梦色蛋糕师', '网球王子']


In [113]:
clust = [78]
suggest(conn, clust, fanju_name)

76
76
14
11
77
390
76
['LoveLiveSchoolIdolProject', 'BanGDream', 'ANIMAYELL', 'LoveLiveSchoolIdolProject第二季', '家庭教师HITMANREBORN']


In [71]:
cursor = conn.cursor()
query = "select * from postgres.ir.distance2 where id1 = {} and id2 = {};".format(1, 2)
cursor.execute(query)
rows = cursor.fetchall()

In [72]:
rows

[(1, 2, 0.0010426838808089843)]

In [27]:
query = '怠惰，486，买苹果吗'

In [28]:
words = jieba.lcut(shorten(query))

In [29]:
for word in words:
    print(word)

怠惰
，
486
，
买苹果


In [43]:
def get_doc_len_file():
    doc_len = dict()
    doc_name = dict()
    f = open("doclen.txt")
    lines = f.readlines()
    doc_len = dict()
    for line in lines:
        split = line.strip().split('-')
        doc_len[int(split[0])] = int(split[1])
    f = open("juji.txt",'r',encoding='UTF-8')
    lines = f.readlines()
    doc_name = dict()
    for line in lines:
        split = line.strip().split(',')
        doc_name[int(split[0])] = split[1][3:]
    return doc_len, doc_name

In [51]:
def get_fanju_file():
    name_fanju = dict()
    fanju_name = dict()
    f = open("dic.txt",'r',encoding='UTF-8')
    lines = f.readlines()
    doc_len = dict()
    for line in lines:
        split = line.strip().split(',')
        fanju_name[int(split[0])] = split[1]
        name_fanju[split[1]] = int(split[0])
    return fanju_name, name_fanju

In [44]:
doc_len, doc_name = get_doc_len_file()

In [53]:
fanju_name, name_fanju = get_fanju_file()

In [55]:
cursor = conn.cursor()
query = "select * from postgres.ir.distance3"
cursor.execute(query)
rows = cursor.fetchall()

In [56]:
len(rows)

1125721

In [58]:
rows[1]

(251, 90, 0.000960510959185561)

In [64]:
matrix = [[0 for i in range(1061)] for j in range(1061)]
for i in rows:
    matrix[i[0]][i[1]] = round(i[2],8)

In [70]:
matrix[1][2]

0.00095033

In [77]:
np.savetxt('dis_matrix_file.txt',matrix, fmt='%.8f') 

In [78]:
import numpy as np

In [79]:
c=np.loadtxt('dis_matrix_file.txt')

In [80]:
c[1][2]

0.00095033

In [132]:
def get_lables(con):
    labels = dict()
    cursor = con.cursor()
    doc_len_query = """select * from postgres.ir.cluster2"""
    cursor.execute(doc_len_query)
    rows = cursor.fetchall()
    cluster = dict()
    for row in rows:
        labels[row[1]] = row[0]
        cluster[row[0]] = []
        for j in row[2][1:-1].split(','):
            cluster[row[0]].append(j)
    return labels, cluster

In [133]:
labels, cluster = get_lables(conn)

In [134]:
cluster

{1: ['0号宿舍',
  '凸变英雄LEAF',
  '小绿和小蓝',
  '我家大师兄脑子有坑特别篇',
  '剑网3·侠肝义胆沈剑心',
  '少年歌行',
  '萌妻食神',
  '通灵妃',
  '全职高手第一季',
  '王者别闹',
  '刺客伍六七',
  '罗小黑战记',
  '如果历史是一群喵',
  '风灵玉秀',
  '灵笼',
  '镇魂街第二季',
  'ReLIFE完结篇',
  '无论何时我们的恋情都是10厘米～告白实行委员会～',
  '从很久以前就喜欢你了～告白实行委员会～',
  '喜欢上你的那个瞬间～告白实行委员会～',
  '姐姐来了',
  '群居姐妹',
  '川柳少女',
  '敦君与女朋友',
  '刀剑神域',
  '刀剑神域ExtraEdition',
  '野良神',
  '野良神ARAGOTO',
  '我们仍未知道那天所看见的花的名字',
  '萤火之森',
  '秒速五厘米',
  '言叶之庭',
  'ACHANNEL',
  '若叶女孩',
  '加奈日记',
  '猫神八百万',
  'OneRoom',
  'OneRoom第二季',
  '牵牛花与加濑同学',
  '玛纳利亚的密友MysteriaFriends巴哈姆特之怒玛娜利亚魔法学院',
  '复活小猫咪',
  '小森同学拒绝不了',
  '房东青春期',
  '猫咪日常',
  '奇异太郎少年的妖怪绘日记',
  '穿越时空的少女',
  '她与她的猫',
  '阳光中的青时雨',
  '八男别闹了',
  '白猫计划零之纪元',
  '格莱普尼尔',
  '神之塔',
  '听我的电波吧',
  '聆听者LISTENERS',
  '昨日之歌',
  '阿尔蒂',
  '公主连结Re',
  '公主连结ReDive',
  '转生成为了只有乙女游戏破灭Flag的邪恶大小姐',
  '辉夜大小姐想让我告白～天才们的恋爱头脑战～',
  '放学后海堤日记',
  '美妙射击部',
  '新樱花大战',
  '社长战斗时间到'],
 2: ['11eyes',
  '出租魔法使',
  '热诚传说X',
  '锁链战记赫克瑟塔斯之光',
  'Caligula卡里古拉',
  '超自然9人组',
  'CHAOS;CHILD',
  '混

In [114]:
def rand(key):
    return random.sample(cluster[key], 5)

In [111]:
rand(1)

['二', '白', 'G', '庭', '她']

In [98]:
import random

In [106]:
def get_lables(con):
    labels = dict()
    cursor = con.cursor()
    doc_len_query = """select * from postgres.ir.cluster2"""
    cursor.execute(doc_len_query)
    rows = cursor.fetchall()
    cluster = dict()
    for row in rows:
        labels[row[1]] = row[0]
        cluster[row[0]] = []
        for j in row[2]:
            cluster[row[0]].append(j)
    return labels, cluster

#cluster = [cluster1.split(','), cluster2.split(','), cluster3.split(','), cluster4.split(','),
#           cluster5.split(','), cluster6.split(','), cluster7.split(','), cluster8.split(','),
#           cluster9.split(','), cluster10.split(','), cluster11.split(','), cluster12.split(','),
#           cluster13.split(','), cluster14.split(','), cluster15.split(','), cluster16.split(','),
#           cluster17.split(',')]

conn = psycopg2.connect(user="checker",
                        password="wangxiaohan",
                        host="121.36.22.150",
                        port="5432",
                        database="postgres")

labels, cluster = get_lables(conn)


def rand(key):
    return random.sample(cluster[key], 5)

In [107]:
rand(1)

['娜', 'm', '道', '小', '是']